In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from nltk.stem.snowball import SnowballStemmer
import warnings; warnings.filterwarnings("ignore");
from sklearn.metrics import mean_squared_error, make_scorer

stemmer = SnowballStemmer('english')
df_train = pd.read_csv('train.csv', encoding="ISO-8859-1") #update here 74067
df_test = pd.read_csv('test.csv', encoding="ISO-8859-1") #update here 166693
df_pro_desc = pd.read_csv('product_descriptions.csv')[:64000] #update here 64000
df_attr = pd.read_csv('attributes.csv')
num_train = df_train.shape[0]

In [2]:
df_brand = df_attr[df_attr.name == "MFG Brand Name"][["product_uid", "value"]].rename(columns={"value": "brand"})
df_brand[0:9]

,product_uid,brand
9,100001.0,Simpson Strong-Tie
37,100002.0,BEHR Premium Textured DeckOver
69,100003.0,STERLING
93,100004.0,Grape Solar
122,100005.0,Delta
163,100006.0,Whirlpool
204,100007.0,Lithonia Lighting
236,100008.0,Teks
256,100009.0,House of Fara


In [3]:
collection = list(df_attr.name)
import re
from fuzzyfinder import fuzzyfinder
def fuzzyfinder(user_input, collection):
        suggestions =[]
        pattern = '.*'.join(user_input) # Converts 'djm' to 'd.*j.*m'
        regex = re.compile(pattern)     # Compiles a regex.
        for item in collection:
            match = regex.search(str(item))  # Checks if the current item matches the regex.
            if match:
                suggestions.append(item)
        return suggestions
material_lower = fuzzyfinder('material', collection)
material_upper = fuzzyfinder('Material', collection)
material_lower.extend(material_upper)

df_materials = df_attr[df_attr['name'].isin(material_lower)][["product_uid", "value"]].rename(columns={"value": "material"})
df_materials=df_materials.drop_duplicates(subset=['product_uid'], keep='first')
df_materials[1:5]

,product_uid,material
67,100003.0,Composite
202,100007.0,Plastic
229,100008.0,Steel
255,100009.0,Medium Density Fiberboard (MDF)


In [4]:
df_type = fuzzyfinder('Type', collection)
df_type_= fuzzyfinder('type', collection)
df_type.extend(df_type_)
df_types = df_attr[df_attr['name'].isin(df_type)][["product_uid", "name"]].rename(columns={"name": "type"})
df_types=df_types.drop_duplicates(subset=['product_uid'], keep='first')
df_types[1:5]

,product_uid,type
66,100003.0,Installation Type
90,100004.0,Electrical Product Type
107,100005.0,Bath Faucet Type
132,100006.0,Appliance Type


In [5]:
def functionfinder(user_input, collection):
        suggestions =[]
        pattern = '.*'.join(user_input) # Converts 'djm' to 'd.*j.*m'
        regex = re.compile(pattern)     # Compiles a regex.
        for item in collection:
            match = regex.search(str(item))  # Checks if the current item matches the regex.
            if match:
                #suggestions.append((match.start(), item))
                suggestions.append(item)
        return suggestions
use = functionfinder('Use', collection)
Resistant = functionfinder('Resistant', collection)
use.extend(Resistant)

df_function = df_attr[df_attr['name'].isin(use)] 
df_function=df_function.rename(columns={"value": "function"})
df_function=df_function.drop_duplicates(subset=['product_uid'], keep='first')
df_function['function_description'] = df_function['name']+" : "+df_function['function']
df_function=df_function.drop(['name','function'],axis=1)
df_function[1:5]

,product_uid,function_description
238,100008.0,Primary Use : Metal to metal fastening
257,100009.0,Moisture Resistant : Yes
376,100012.0,Stain Resistant : No
452,100017.0,Corrosion Resistant : Yes


In [6]:
num_train = df_train.shape[0] #74067
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True) #train+test zongxiangdiejia 240760
df_all = pd.merge(df_all, df_pro_desc, how='left', on='product_uid')
df_all = pd.merge(df_all, df_brand, how='left', on='product_uid') #240760
df_all = pd.merge(df_all, df_materials, how='left', on='product_uid')
df_all = pd.merge(df_all, df_function, how='left', on='product_uid')
df_all = pd.merge(df_all, df_types, how='left', on='product_uid')
df_all[0:5]

,id,product_title,product_uid,relevance,search_term,product_description,brand,material,function_description,type
0,2,Simpson Strong-Tie 12-Gauge Angle,100001,3.00,angle bracket,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie,Galvanized Steel,NaN,NaN
1,3,Simpson Strong-Tie 12-Gauge Angle,100001,2.50,l bracket,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie,Galvanized Steel,NaN,NaN
2,9,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,100002,3.00,deck over,BEHR Premium Textured DECKOVER is an innovativ...,BEHR Premium Textured DeckOver,NaN,Concrete Use : Yes,Paint Product Type
3,16,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.33,rain shower head,Update your bathroom with the Delta Vero Singl...,Delta,NaN,NaN,Bath Faucet Type
4,17,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.67,shower only faucet,Update your bathroom with the Delta Vero Singl...,Delta,NaN,NaN,Bath Faucet Type


In [7]:
strNum = {'zero':0,'one':1,'two':2,'three':3,'four':4,'five':5,'six':6,'seven':7,'eight':8,'nine':9}
def str_stem(s): 
    if isinstance(s,str):
        s = re.sub(r"(\w)\.([A-Z])", r"\1 \2", s) #Split words with a.A
        s = s.lower()
        s = s.replace("  "," ")
        s = s.replace(",","") #could be number / segment later
        s = s.replace("$"," ")
        s = s.replace("?"," ")
        s = s.replace("-"," ")
        s = s.replace("//","/")
        s = s.replace("..",".")
        s = s.replace(" / "," ")
        s = s.replace(" \\ "," ")
        s = s.replace("."," . ")
        s = re.sub(r"(^\.|/)", r"", s)
        s = re.sub(r"(\.|/)$", r"", s)
        
        s = s.replace("  "," ")
        s = s.replace(" . "," ")
        #s = (" ").join([z for z in s.split(" ") if z not in stop_w])
        s = (" ").join([str(strNum[z]) if z in strNum else z for z in s.split(" ")])
        s = (" ").join([stemmer.stem(z) for z in s.split(" ")])
        
        s = s.lower()
        s = s.replace("toliet","toilet")
        s = s.replace("airconditioner","air conditioner")
        s = s.replace("vinal","vinyl")
        s = s.replace("vynal","vinyl")
        s = s.replace("skill","skil")
        s = s.replace("snowbl","snow bl")
        s = s.replace("plexigla","plexi gla")
        s = s.replace("rustoleum","rust-oleum")
        s = s.replace("whirpool","whirlpool")
        s = s.replace("whirlpoolga", "whirlpool ga")
        s = s.replace("whirlpoolstainless","whirlpool stainless")
        return s
    else:
        return "null"

In [8]:
def str_common_word(str1, str2):
	return sum(int(str2.find(word)>=0) for word in str1.split())

In [9]:
df_all[1:3]

,id,product_title,product_uid,relevance,search_term,product_description,brand,material,function_description,type
1,3,Simpson Strong-Tie 12-Gauge Angle,100001,2.5,l bracket,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie,Galvanized Steel,NaN,NaN
2,9,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,100002,3.0,deck over,BEHR Premium Textured DECKOVER is an innovativ...,BEHR Premium Textured DeckOver,NaN,Concrete Use : Yes,Paint Product Type


In [10]:
df_all['brand'] = df_all['brand'].map(lambda x:str_stem(x))

In [11]:
df_all['type'] = df_all['type'].map(lambda x:str_stem(x))

In [12]:
df_all['search_term'] = df_all['search_term'].map(lambda x:str_stem(x)) #map(function, sequence)
df_all['product_title'] = df_all['product_title'].map(lambda x:str_stem(x))
df_all['product_description'] = df_all['product_description'].map(lambda x:str_stem(x))
#df_all['brand'] = df_all['brand'].map(lambda x:str_stem(x))
df_all['material'] = df_all['material'].map(lambda x:str_stem(x))
df_all['function_description'] = df_all['function_description'].map(lambda x:str_stem(x))

In [13]:
df_all['product_info'] = df_all['search_term']+"\t"+df_all['product_title'] +"\t"+df_all['product_description']
df_all['len_of_query'] = df_all['search_term'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_title'] = df_all['product_title'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_description'] = df_all['product_description'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_brand'] = df_all['brand'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_types'] = df_all['type'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_function_description'] = df_all['function_description'].map(lambda x:len(x.split())).astype(np.int64)
#df_all['search_term'] = df_all['product_info'].map(lambda x:seg_words(x.split('\t')[0],x.split('\t')[1]))
df_all[0:5]

,id,product_title,product_uid,relevance,search_term,product_description,brand,material,function_description,type,product_info,len_of_query,len_of_title,len_of_description,len_of_brand,len_of_types,len_of_function_description
0,2,simpson strong tie 12 gaug angl,100001,3.00,angl bracket,not onli do angl make joint stronger they also...,simpson strong tie,galvan steel,null,null,angl bracket\tsimpson strong tie 12 gaug angl\...,2,6,138,3,1,1
1,3,simpson strong tie 12 gaug angl,100001,2.50,l bracket,not onli do angl make joint stronger they also...,simpson strong tie,galvan steel,null,null,l bracket\tsimpson strong tie 12 gaug angl\tno...,2,6,138,3,1,1
2,9,behr premium textur deckov 1 gal #sc 141 tugbo...,100002,3.00,deck over,behr premium textur deckov is an innov solid c...,behr premium textur deckov,null,concret use : yes,paint product type,deck over\tbehr premium textur deckov 1 gal #s...,2,13,171,4,3,4
3,16,delta vero 1 handl shower onli faucet trim kit...,100005,2.33,rain shower head,updat your bathroom with the delta vero singl ...,delta,null,null,bath faucet type,rain shower head\tdelta vero 1 handl shower on...,3,14,109,1,3,1
4,17,delta vero 1 handl shower onli faucet trim kit...,100005,2.67,shower onli faucet,updat your bathroom with the delta vero singl ...,delta,null,null,bath faucet type,shower onli faucet\tdelta vero 1 handl shower ...,3,14,109,1,3,1


In [14]:
df_all['query_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
df_all['query_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))
df_all['query_last_word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0].split(" ")[-1],x.split('\t')[1]))
df_all['query_last_word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0].split(" ")[-1],x.split('\t')[2]))

df_all['word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
df_all['word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))

In [15]:
df_all['ratio_title'] = df_all['word_in_title']/df_all['len_of_query']
df_all['ratio_description'] = df_all['word_in_description']/df_all['len_of_query']
#df_all['attr_brand'] = df_all['search_term']+"\t"+df_all['brand']
#df_all['attr_material'] = df_all['search_term']+"\t"+df_all['material']
#df_all['attr_function'] = df_all['search_term']+"\t"+df_all['function_description']
df_all['attr'] = df_all['search_term']+"\t"+df_all['brand']+"\t"+df_all['material']+"\t"+df_all['function_description']+"\t"+df_all['type']
df_all[0:5]

,id,product_title,product_uid,relevance,search_term,product_description,brand,material,function_description,type,...,len_of_function_description,query_in_title,query_in_description,query_last_word_in_title,query_last_word_in_description,word_in_title,word_in_description,ratio_title,ratio_description,attr
0,2,simpson strong tie 12 gaug angl,100001,3.00,angl bracket,not onli do angl make joint stronger they also...,simpson strong tie,galvan steel,null,null,...,1,1,1,0,0,1,1,0.500000,0.500000,angl bracket\tsimpson strong tie\tgalvan steel...
1,3,simpson strong tie 12 gaug angl,100001,2.50,l bracket,not onli do angl make joint stronger they also...,simpson strong tie,galvan steel,null,null,...,1,1,1,0,0,1,1,0.500000,0.500000,l bracket\tsimpson strong tie\tgalvan steel\tn...
2,9,behr premium textur deckov 1 gal #sc 141 tugbo...,100002,3.00,deck over,behr premium textur deckov is an innov solid c...,behr premium textur deckov,null,concret use : yes,paint product type,...,4,1,1,0,0,1,1,0.500000,0.500000,deck over\tbehr premium textur deckov\tnull\tc...
3,16,delta vero 1 handl shower onli faucet trim kit...,100005,2.33,rain shower head,updat your bathroom with the delta vero singl ...,delta,null,null,bath faucet type,...,1,1,1,0,0,1,1,0.333333,0.333333,rain shower head\tdelta\tnull\tnull\tbath fauc...
4,17,delta vero 1 handl shower onli faucet trim kit...,100005,2.67,shower onli faucet,updat your bathroom with the delta vero singl ...,delta,null,null,bath faucet type,...,1,3,3,1,1,3,3,1.000000,1.000000,shower onli faucet\tdelta\tnull\tnull\tbath fa...


In [16]:
df_all['word_in_brand'] = df_all['attr'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))#search_term, brand
df_all['word_in_material'] = df_all['attr'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))
df_all['word_in_function_description'] = df_all['attr'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[3]))
df_all['ratio_brand'] = df_all['word_in_brand']/df_all['len_of_brand']
df_all['len_of_material'] = df_all['material'].map(lambda x:len(x.split())).astype(np.int64)
df_all['ratio_material'] = df_all['word_in_material']/df_all['len_of_material']
df_all['ratio_function'] = df_all['word_in_function_description']/df_all['len_of_function_description']
df_all['word_in_type'] = df_all['attr'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[4]))
df_all['ratio_type'] = df_all['word_in_type']/df_all['len_of_types']
df_brand = pd.unique(df_all.brand.ravel()) #brand变为array后挑出唯一的brand name
#df_materials = pd.unique(df_all.material.ravel())
#df_function = pd.unique(df_all.function_description.ravel())
#df_materials = pd.unique(df_all.material.ravel())
df_all[0:2]

,id,product_title,product_uid,relevance,search_term,product_description,brand,material,function_description,type,...,attr,word_in_brand,word_in_material,word_in_function_description,ratio_brand,len_of_material,ratio_material,ratio_function,word_in_type,ratio_type
0,2,simpson strong tie 12 gaug angl,100001,3.0,angl bracket,not onli do angl make joint stronger they also...,simpson strong tie,galvan steel,null,null,...,angl bracket\tsimpson strong tie\tgalvan steel...,0,0,0,0.0,2,0.0,0.0,0,0.0
1,3,simpson strong tie 12 gaug angl,100001,2.5,l bracket,not onli do angl make joint stronger they also...,simpson strong tie,galvan steel,null,null,...,l bracket\tsimpson strong tie\tgalvan steel\tn...,0,1,1,0.0,2,0.5,1.0,1,1.0


In [17]:
df_all = df_all.drop(['search_term','product_title','product_description','product_info'],axis=1)
df_all[1:2]

,id,product_uid,relevance,brand,material,function_description,type,len_of_query,len_of_title,len_of_description,...,attr,word_in_brand,word_in_material,word_in_function_description,ratio_brand,len_of_material,ratio_material,ratio_function,word_in_type,ratio_type
1,3,100001,2.5,simpson strong tie,galvan steel,null,null,2,6,138,...,l bracket\tsimpson strong tie\tgalvan steel\tn...,0,1,1,0.0,2,0.5,1.0,1,1.0


In [18]:
df_all = df_all.drop(['brand','material','type','function_description'],axis=1)
df_all = df_all.drop(['attr'],axis=1)

df_all[1:2]

,id,product_uid,relevance,len_of_query,len_of_title,len_of_description,len_of_brand,len_of_types,len_of_function_description,query_in_title,...,ratio_description,word_in_brand,word_in_material,word_in_function_description,ratio_brand,len_of_material,ratio_material,ratio_function,word_in_type,ratio_type
1,3,100001,2.5,2,6,138,3,1,1,1,...,0.5,0,1,1,0.0,2,0.5,1.0,1,1.0


In [19]:
df_train = df_all.iloc[:num_train]
df_test = df_all.iloc[num_train:]
id_test = df_test['id']

y_train = df_train['relevance'].values

#X_train = df_train[:]
X_train = df_train.drop(['id','relevance'],axis=1).values

#X_test = df_test[:]
X_test = df_test.drop(['id','relevance'],axis=1).values
#X_train = df_train[:]
#X_test = df_test[:]

In [20]:
X_train=X_train[:,1:]
X_test=X_test[:,1:]

In [21]:
np.any(np.isnan(X_train)) #true


False

In [22]:
np.all(np.isfinite(X_train))#flase

True

In [23]:
from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from nltk.stem.snowball import SnowballStemmer
rf = RandomForestRegressor(n_estimators=15, max_depth=6, random_state=0)
clf = BaggingRegressor(rf, n_estimators=45, max_samples=0.1, random_state=25)
#imp = Imputer(missing_values='NaN', strategy='most_frequent')
#imp.fit_transform(X_train)
#imp.fit_transform(y_train)
clf.fit(X_train, y_train)
y_predrf = clf.predict(X_test)
len(y_predrf)


166693

In [116]:
from sklearn.neural_network import MLPRegressor
nn = MLPRegressor(hidden_layer_sizes=(100, 3), alpha=1e-5, random_state=1, batch_size=200) #好多变量可以试
nn.fit(X_train, y_train)
y_prednn = nn.predict(X_test)
len(y_prednn)

166693

In [125]:
y_pred = 0.7*y_predrf +0.3*y_prednn
df_sol = pd.read_csv('solution.csv', encoding="ISO-8859-1")
df_sol['pred'] = y_pred
df_fliter = df_sol[df_sol.Usage == "Public"]

In [126]:
from sklearn.metrics import mean_squared_error, make_scorer
mse = mean_squared_error(df_fliter.relevance, df_fliter.pred, sample_weight=None, multioutput='uniform_average')
np.sqrt(mse)

0.48323758314690268

In [27]:
from sklearn.metrics import mean_absolute_error
MAE = mean_absolute_error(df_fliter.relevance, df_fliter.pred, sample_weight=None, multioutput='uniform_average')
np.sqrt(MAE)

0.62465512922302668

In [28]:
from sklearn.model_selection import cross_val_score
score_rf = cross_val_score(clf,X_train,y_train,cv=5,scoring='mean_squared_error')
score_nn = cross_val_score(nn,X_train,y_train,cv=5,scoring='mean_squared_error')
score = 0.5*(score_rf + score_nn)
score.mean()

-0.23889591370012209